In [1]:
%pip install selenium

  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached trio-0.30.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached certifi-2025.7.14-py3-none-any.whl.metadata (2.4 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached cffi-1.17.1-cp313-cp313-win_amd64.whl.metadata (1.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)

In [2]:
%pip install webdriver-manager

  Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl.metadata (12 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached python_dotenv-1.1.1-py3-none-any.whl.metadata (24 kB)
  Using cached charset_normalizer-3.4.2-cp313-cp313-win_amd64.whl.metadata (36 kB)
Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl (27 kB)
Using cached python_dotenv-1.1.1-py3-none-any.whl (20 kB)
Using cached requests-2.32.4-py3-none-any.whl (64 kB)
Using cached charset_normalizer-3.4.2-cp313-cp313-win_amd64.whl (105 kB)

   ---------------------------------------- 4/4 [webdriver-manager]

Note: you may need to restart the kernel to use updated packages.


In [2]:
import time
import json
import os
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 1. 크롬 드라이버 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.hyundai.com/kr/ko/e/all-vehicles")
driver.maximize_window()
time.sleep(3)

# 2. 데이터 저장용 리스트
car_data = []

# 이미지 저장 폴더 생성
base_folder = "downloaded_images"
os.makedirs(base_folder, exist_ok=True)

# 3. category-item 블록들 찾기
category_items = driver.find_elements(By.CLASS_NAME, "category-item")

for cat_idx, category in enumerate(category_items):
    try:
        # 자동차 종류
        category_title = category.find_element(By.CLASS_NAME, "txt").text.strip().replace("/", "_").replace(" ", "_")
        category_folder = os.path.join(base_folder, category_title)
        os.makedirs(category_folder, exist_ok=True)
        
        # 자동차 이미지 - 자동차 이름
        vehicle_items = category.find_elements(By.CLASS_NAME, "item")
        
        for idx, item in enumerate(vehicle_items):
            try:
                img_tag = item.find_element(By.CLASS_NAME, "el-image").find_element(By.TAG_NAME, "img")
                img_src = img_tag.get_attribute("src")
                img_alt = img_tag.get_attribute("alt").strip().replace("/", "_").replace(" ", "_")
                
                # 이미지 다운로드
                response = requests.get(img_src)
                if response.status_code == 200:
                    img_ext = os.path.splitext(img_src)[-1].split("?")[0]  # 확장자 (.png, .jpg 등)
                    img_filename = f"{idx+1}_{img_alt}{img_ext}"
                    img_path = os.path.join(category_folder, img_filename)
                    
                    with open(img_path, "wb") as f:
                        f.write(response.content)
                    
                    print(f"✅ {category_title} | {img_alt} 이미지 다운로드 완료")
                else:
                    print(f"⚠️ {category_title} | {img_alt} 이미지 다운로드 실패 (status {response.status_code})")

                car_data.append({
                    "category": category_title,
                    "car_name": img_alt,
                    "image_src": img_src
                })
            
            except Exception as e:
                print(f"⚠️ {category_title} 내 차량 처리 오류: {e}")
                
    except Exception as e:
        print(f"❌ 카테고리 {cat_idx+1} 처리 오류: {e}")

driver.quit()

# 4. JSON 저장
with open("hyundai_car_data.json", "w", encoding="utf-8") as f:
    json.dump(car_data, f, ensure_ascii=False, indent=2)

print("✅ 최종 저장 완료: hyundai_car_data.json + 이미지 파일들")


✅ 수소___전기차 | 디_올_뉴_넥쏘 이미지 다운로드 완료
✅ 수소___전기차 | 아이오닉_5 이미지 다운로드 완료
✅ 수소___전기차 | 코나_Electric 이미지 다운로드 완료
✅ 수소___전기차 | 아이오닉_9 이미지 다운로드 완료
✅ 수소___전기차 | ST1 이미지 다운로드 완료
✅ 수소___전기차 | 아이오닉_6 이미지 다운로드 완료
✅ 수소___전기차 | 포터_II_Electric 이미지 다운로드 완료
✅ 수소___전기차 | 포터_II_Electric_특장차 이미지 다운로드 완료
✅ 수소___전기차 | 2026_캐스퍼_일렉트릭 이미지 다운로드 완료
✅ N | 아반떼_N 이미지 다운로드 완료
✅ N | 아이오닉_5_N 이미지 다운로드 완료
✅ 승용 | 그랜저 이미지 다운로드 완료
✅ 승용 | 그랜저_Hybrid 이미지 다운로드 완료
✅ 승용 | 아반떼 이미지 다운로드 완료
✅ 승용 | 아반떼_Hybrid 이미지 다운로드 완료
✅ 승용 | 쏘나타_디_엣지 이미지 다운로드 완료
✅ 승용 | 쏘나타_디_엣지_Hybrid 이미지 다운로드 완료
✅ SUV | 코나 이미지 다운로드 완료
✅ SUV | 코나_Hybrid 이미지 다운로드 완료
✅ SUV | 베뉴 이미지 다운로드 완료
✅ SUV | 디_올_뉴_팰리세이드 이미지 다운로드 완료
✅ SUV | 디_올_뉴_팰리세이드_Hybrid 이미지 다운로드 완료
✅ SUV | 투싼 이미지 다운로드 완료
✅ SUV | 투싼_Hybrid 이미지 다운로드 완료
✅ SUV | 싼타페 이미지 다운로드 완료
✅ SUV | 싼타페_Hybrid 이미지 다운로드 완료
✅ SUV | 2026_캐스퍼 이미지 다운로드 완료
✅ MPV | 스타리아_라운지 이미지 다운로드 완료
✅ MPV | 스타리아_라운지_Hybrid 이미지 다운로드 완료
✅ MPV | 스타리아 이미지 다운로드 완료
✅ MPV | 스타리아_Hybrid 이미지 다운로드 완료
✅ MPV | 스타리아_킨더 이미지 다운로드 완료
✅ MPV | 스타리아_라운지_캠퍼 이미지 다운로